In [1]:
import os
import sys 

import pandas as pd
import numpy as np 
import geopandas as gpd
import ee
import geemap
from tqdm import tqdm_notebook

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
dataset = ee.ImageCollection('WRI/Aqueduct_Flood_Hazard_Maps/V2')

# Get distinct values for the "model" property
distinct_models = dataset.distinct('model')

# Reduce to a list of the "model" property values
model_values = distinct_models.aggregate_array('model').getInfo()

# Print the list of distinct model values
print("Distinct values for 'model':", model_values)

Distinct values for 'model': ['000000000WATCH', '00000NorESM1-M', '0000GFDL-ESM2M', '0000HadGEM2-ES', '00IPSL-CM5A-LR', 'MIROC-ESM-CHEM']


In [52]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
poly = poly.to_crs(epsg=4326)
gid_2 = poly['GID_2'].to_list()
flood_depth = []

# Define the dataset collection
# https://developers.google.com/earth-engine/datasets/catalog/WRI_Aqueduct_Flood_Hazard_Maps_V2#image-properties
# https://files.wri.org/d8/s3fs-public/aqueduct-floods-methodology.pdf?_gl=1*1u9ay0w*_gcl_au*MTg3NzAwNTI0OC4xNzI0MjQwNTIw
# https://www.usgs.gov/centers/new-jersey-water-science-center/floods-recurrence-intervals-and-100-year-floods 
flood = ee.ImageCollection('WRI/Aqueduct_Flood_Hazard_Maps/V2').filter(ee.Filter.eq('climatescenario', 'rcp8p5')) \
                                                         .filter(ee.Filter.eq('floodtype', 'inunriver')) \
                                                         .filter(ee.Filter.eq('returnperiod', 100))\
                                                         .filter(ee.Filter.eq('model', '0000GFDL-ESM2M'))\
                                                         .filter(ee.Filter.eq('year', 2030))\
                                                         .sum()\
                                                         .select('inundation_depth')

for i in tqdm_notebook(range(len(poly))):

    r = poly.iloc[i:i+1]
    region = geemap.gdf_to_ee(r).geometry()

    stats = flood.reduceRegion(
                      geometry=region,
                      reducer=ee.Reducer.mean(),
                      scale=1000,  
                      crs='EPSG:4326', 
                      tileScale=14,
                      bestEffort=True
                )
    
    
    try:
        depth = stats.getInfo()['inundation_depth']
        flood_depth.append(depth)
        
    except Exception as e:
         print(e)
         continue
    
flood_df = pd.DataFrame({'GID_2': gid_2,
                         'inundation_depth': flood_depth})
flood_df['year'] = 2022 
    

C:\Users\sonle\AppData\Local\Temp\ipykernel_1736\3592506215.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(poly))):


  0%|          | 0/161 [00:00<?, ?it/s]

In [59]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_0.json')
region = geemap.gdf_to_ee(poly).geometry()

# Define the dataset collection
for year in [2030, 2050, 2080]:
    flood = ee.ImageCollection('WRI/Aqueduct_Flood_Hazard_Maps/V2').filter(ee.Filter.eq('climatescenario', 'rcp8p5')) \
                                                            .filter(ee.Filter.eq('floodtype', 'inunriver')) \
                                                            .filter(ee.Filter.eq('returnperiod', 100))\
                                                            .filter(ee.Filter.eq('model', '0000GFDL-ESM2M'))\
                                                            .filter(ee.Filter.eq('year', year))\
                                                            .sum()\
                                                            .select('inundation_depth')


    task = ee.batch.Export.image.toDrive(
        image=flood,
        description=f'SRB_{year}_1in100_rcp8p5',
        folder='wri_flooding',
        region=region,
        scale=1000,
        crs='EPSG:4326',
        fileNamePrefix=f'SRB_{year}_1in100_rcp8p5',
        # maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task.start()

In [61]:
from datetime import datetime

from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

In [62]:
def query_osm_overpass(area_name, key, value):

    nominatim = Nominatim()
    areaId = nominatim.query(area_name).areaId()
    overpass = Overpass()
    query = overpassQueryBuilder(area=areaId, elementType='way', selector=f'"{key}"="{value}"', out='body', includeGeometry=True)
    result = overpass.query(query, timeout=120)
    
    return result



def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

def merge_with_connectivity(out, ookla):
    out = out[~out.lat.isna()]
    out['quadkey'] = out.apply(make_quadkey, axis=1)
    out['quadkey'] = out['quadkey'].astype('str')
    connectivity = ookla[ookla.quadkey.isin(out['quadkey'])]
    connectivity = connectivity.set_index('quadkey')
    tmp = out.set_index('quadkey')
    tmp.index = tmp.index.astype('str')
    tmp = tmp.join(connectivity, how='left')
    tmp = tmp.reset_index().drop_duplicates(
        subset=['name', 'quadkey'],
        keep='last').reset_index(drop=True)
    # return connectivity
    return tmp

def make_quadkey(row):
    return qk.from_geo((row['lat'], row['lon']), 16)

def make_details(element):
    tags = element.tags()
    if 'name' in tags:
        out = dict()
        out['amenity'] = tags['amenity']
        out['name'] = tags['name']
        if 'lat' in tags:
            out['lat'] = tags['lat']
            out['lon'] = tags['lon']
        elif bool(element.centerLat()):
            out['lat'] = element.centerLat()
            out['lon'] = element.centerLon()
        elif bool(element.geometry()):
            out['lat'] = element.geometry()['coordinates'][0][0][1]
            out['lon'] = element.geometry()['coordinates'][0][0][0]
        else:
            out['lat'] = None
            out['lon'] = None
        return out
    else:
        return None
    
def filter_elements(result):
    elements = list()
    for element in result.elements():
        elements.append(make_details(element))
    filtered = [element for element in elements if element is not None]
    return filtered

In [63]:
result = query_osm_overpass('Serbia', 'highway', 'motorway')

[overpass] downloading data: [timeout:120][out:json];area(3609088937)->.searchArea;(way["highway"="motorway"](area.searchArea);); out body geom;


In [68]:
result.elements()[10].tags()

{'highway': 'motorway',
 'int_ref': 'E 70',
 'lanes': '3',
 'lit': 'yes',
 'maxspeed': '40',
 'note': 'ZA POČETNIKE: Naplatna rampa je mapirana ispravno, sa svim propratnim podacima: toll, lanes, maxspeed, itd. Nemojte docrtavati puteve kao posebne trake. Nije ispravno.',
 'oneway': 'yes',
 'placement': 'right_of:1',
 'ref': 'A3',
 'surface': 'asphalt'}

In [70]:
roads = gpd.read_file("D:\Work\WB\LDT\countries\SRB\\raw_data\serbia-230101-free.shp\gis_osm_roads_free_1.shp")
roads

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4432186,5122,residential,Fő utca,None,B,0,0,F,F,"LINESTRING (20.01588 46.17707, 20.01869 46.178..."
1,4759407,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.65914 42.87117, 22.65911 42.871..."
2,4759488,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.6802 42.86464, 22.68017 42.8646..."
3,4907997,5113,primary,Саве Ковачевића,12,B,50,0,F,F,"LINESTRING (20.4841 45.42493, 20.4831 45.42412..."
4,4907998,5114,secondary,Бирчанинова,None,B,0,0,F,F,"LINESTRING (20.40807 45.37685, 20.40849 45.37722)"
...,...,...,...,...,...,...,...,...,...,...,...
453627,1126353054,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83059 45.25472, 19.83073 45.254..."
453628,1126353055,5122,residential,Боре Продановића,None,B,0,0,F,F,"LINESTRING (19.83059 45.25472, 19.83062 45.254..."
453629,1126353056,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83058 45.25477, 19.83045 45.25476)"
453630,1126353057,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83045 45.25476, 19.8304 45.2547..."


In [71]:
roads['fclass'].value_counts()

fclass
residential       126497
service            95994
track              83474
footway            37583
unclassified       33946
secondary          11618
path               11110
tertiary            9743
track_grade3        9722
primary             7926
steps               4353
motorway            4146
track_grade4        3562
track_grade2        2077
motorway_link       1988
cycleway            1797
living_street       1792
track_grade5        1655
pedestrian          1361
primary_link        1118
secondary_link       713
track_grade1         646
trunk                394
tertiary_link        246
trunk_link           125
bridleway             45
unknown                1
Name: count, dtype: int64

In [72]:
valid_roads = ['residential', 'service', 'track', 'unclassified', 'secondary', 'tertiary', 'track_grade3', 'primary',
               'motorway', 'track_grade2', 'track_grade4', 'track_grade5', 'track_grade1', 'motorway_link', 'living_street',
               'primary_link', 'secondary_link', 'tertiary_link', 'trunk', 'trunk_link']

roads[roads['fclass'].isin(valid_roads)]

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4432186,5122,residential,Fő utca,None,B,0,0,F,F,"LINESTRING (20.01588 46.17707, 20.01869 46.178..."
1,4759407,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.65914 42.87117, 22.65911 42.871..."
2,4759488,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.6802 42.86464, 22.68017 42.8646..."
3,4907997,5113,primary,Саве Ковачевића,12,B,50,0,F,F,"LINESTRING (20.4841 45.42493, 20.4831 45.42412..."
4,4907998,5114,secondary,Бирчанинова,None,B,0,0,F,F,"LINESTRING (20.40807 45.37685, 20.40849 45.37722)"
...,...,...,...,...,...,...,...,...,...,...,...
453627,1126353054,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83059 45.25472, 19.83073 45.254..."
453628,1126353055,5122,residential,Боре Продановића,None,B,0,0,F,F,"LINESTRING (19.83059 45.25472, 19.83062 45.254..."
453629,1126353056,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83058 45.25477, 19.83045 45.25476)"
453630,1126353057,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83045 45.25476, 19.8304 45.2547..."


In [82]:
valid_roads_gdf = roads[roads['fclass'].isin(valid_roads)]
valid_roads_gdf.to_file('D:\Work\WB\LDT\countries\SRB\shapefiles/roads.json', driver='GeoJSON', index=False)

In [92]:
valid_roads_gdf = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles/roads.json')
valid_roads_gdf

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4432186,5122,residential,Fő utca,None,B,0,0,F,F,"LINESTRING (20.01588 46.17707, 20.01869 46.178..."
1,4759407,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.65914 42.87117, 22.65911 42.871..."
2,4759488,5122,residential,None,None,B,0,0,F,F,"LINESTRING (22.6802 42.86464, 22.68017 42.8646..."
3,4907997,5113,primary,Саве Ковачевића,12,B,50,0,F,F,"LINESTRING (20.4841 45.42493, 20.4831 45.42412..."
4,4907998,5114,secondary,Бирчанинова,None,B,0,0,F,F,"LINESTRING (20.40807 45.37685, 20.40849 45.37722)"
...,...,...,...,...,...,...,...,...,...,...,...
397377,1126353054,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83059 45.25472, 19.83073 45.254..."
397378,1126353055,5122,residential,Боре Продановића,None,B,0,0,F,F,"LINESTRING (19.83059 45.25472, 19.83062 45.254..."
397379,1126353056,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83058 45.25477, 19.83045 45.25476)"
397380,1126353057,5115,tertiary,Новосадског Сајма,None,F,50,0,F,F,"LINESTRING (19.83045 45.25476, 19.8304 45.2547..."


In [91]:
rails = gpd.read_file("D:\Work\WB\LDT\countries\SRB\\raw_data\serbia-230101-free.shp\gis_osm_railways_free_1.shp")
rails

,osm_id,code,fclass,name,layer,bridge,tunnel,geometry
0,8092168,6101,rail,None,0,F,F,"LINESTRING (19.67441 45.45825, 19.67423 45.458..."
1,9849396,6101,rail,None,0,F,F,"LINESTRING (20.3654 44.83872, 20.36485 44.8389..."
2,9849639,6101,rail,None,1,T,F,"LINESTRING (20.3534 44.84077, 20.35291 44.84106)"
3,9849660,6101,rail,None,0,F,F,"LINESTRING (20.35699 44.83917, 20.35639 44.839..."
4,9849728,6101,rail,None,1,T,F,"LINESTRING (20.35753 44.83907, 20.35699 44.83917)"
...,...,...,...,...,...,...,...,...
6586,1124896883,6101,rail,Краљево-Чачак-Пожега,1,T,F,"LINESTRING (20.09813 43.86638, 20.09805 43.86635)"
6587,1125063588,6101,rail,Београд - Бар,1,F,F,"LINESTRING (19.99965 43.87732, 20 43.87609, 20..."
6588,1125063589,6101,rail,Београд - Бар,1,T,F,"LINESTRING (20.00506 43.87151, 20.00516 43.87147)"
6589,1125093117,6101,rail,Београд - Бар,0,F,F,"LINESTRING (20.00541 43.81009, 20.00481 43.810..."


In [93]:
import rasterio
from rasterstats import zonal_stats

ds = rasterio.open("D:\Work\WB\LDT\countries\SRB\\raw_data\SRB_2080_1in100_rcp8p5.tif")
arr = ds.read(1)
affine = ds.transform

zonal_rails = zonal_stats(rails.geometry, arr, affine=affine, stats='mean')
rails['flood_inundation_depth'] = pd.DataFrame(zonal_rails)

zonal_roads = zonal_stats(valid_roads_gdf.geometry, arr, affine=affine, stats='mean')
valid_roads_gdf['flood_inundation_depth'] = pd.DataFrame(zonal_roads)


In [94]:
valid_roads_gdf = valid_roads_gdf[~valid_roads_gdf['flood_inundation_depth'].isnull()]
rails = rails[~rails['flood_inundation_depth'].isnull()]

In [119]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')

In [120]:
roads_flood = valid_roads_gdf[valid_roads_gdf['flood_inundation_depth'] >= 0.3]
roads_flood = roads_flood.to_crs(epsg=3857)
roads_flood['length'] = roads_flood.length
roads_flood = roads_flood.to_crs(epsg=4326)
roads_flood = roads_flood.sjoin(poly[['GID_2', 'NAME_2', 'geometry']])
poly = poly.merge(roads_flood.groupby(['GID_2']).agg({'length': 'sum'}).reset_index(), 
                  on=['GID_2'], how='left')
poly['length'] = poly['length'].fillna(0)
poly = poly.rename({'length': 'road_length_flood_risk'}, axis=1)

rails_flood = rails[rails['flood_inundation_depth'] >= 0.3]
rails_flood = rails_flood.to_crs(epsg=3857)
rails_flood['length'] = rails_flood.length
rails_flood = rails_flood.to_crs(epsg=4326)
rails_flood = rails_flood.sjoin(poly[['GID_2', 'NAME_2', 'geometry']])
poly = poly.merge(rails_flood.groupby(['GID_2']).agg({'length': 'sum'}).reset_index(), 
                  on=['GID_2'], how='left')
poly['length'] = poly['length'].fillna(0)
poly = poly.rename({'length': 'railway_length_flood_risk'}, axis=1)



pd.DataFrame(poly[['GID_2', 'railway_length_flood_risk', 'road_length_flood_risk']]).to_csv('D:\Work\WB\LDT\countries\SRB\datasets\SRB_flood_risk_2022.csv', index=False)

In [9]:
flood = pd.read_csv('D:\Work\WB\LDT\countries\SRB\datasets\SRB_flood_risk_2022.csv')

In [3]:

poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
poly = poly.to_crs(epsg=4326)
gid_2 = poly['GID_2'].to_list()
pop_stats = []

# Define the dataset collection
# https://developers.google.com/earth-engine/datasets/catalog/WRI_Aqueduct_Flood_Hazard_Maps_V2#image-properties
# https://files.wri.org/d8/s3fs-public/aqueduct-floods-methodology.pdf?_gl=1*1u9ay0w*_gcl_au*MTg3NzAwNTI0OC4xNzI0MjQwNTIw
# https://www.usgs.gov/centers/new-jersey-water-science-center/floods-recurrence-intervals-and-100-year-floods 
pops = ee.ImageCollection("projects/sat-io/open-datasets/ORNL/LANDSCAN_GLOBAL").filterDate('2022-01-01', '2022-12-31')\
                                                         .sum()\
                                                         .select('b1')

for i in tqdm_notebook(range(len(poly))):

    r = poly.iloc[i:i+1]
    region = geemap.gdf_to_ee(r).geometry()

    stats = pops.reduceRegion(
                      geometry=region,
                      reducer=ee.Reducer.sum(),
                      scale=1000,  
                      crs='EPSG:4326', 
                      tileScale=14,
                      bestEffort=True
                )
    
    
    try:
        pop = stats.getInfo()['b1']
        pop_stats.append(pop)
        
    except Exception as e:
         print(e)
         continue
    
pop_df = pd.DataFrame({'GID_2': gid_2,
                       'population_sum': pop_stats})
pop_df['year'] = 2022 
    

C:\Users\sonle\AppData\Local\Temp\ipykernel_22152\3562547755.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(poly))):


  0%|          | 0/161 [00:00<?, ?it/s]

In [10]:
flood['year'] = 2022
flood['railway_length_flood_risk'] = flood['railway_length_flood_risk'] 
flood['road_length_flood_risk'] = flood['road_length_flood_risk'] 

flood = pd.merge(flood, pop_df, on=['GID_2', 'year'], how='left')

flood['rail_length_flood_pcap'] = np.round(flood['railway_length_flood_risk'] / flood['population_sum'], 3)
flood['road_length_flood_pcap'] = np.round(flood['road_length_flood_risk'] / flood['population_sum'], 3)

flood

,GID_2,railway_length_flood_risk,road_length_flood_risk,year,population_sum,rail_length_flood_pcap,road_length_flood_pcap
0,SRB.1.1_1,0.000000,0.000000,2022,36938.533333,0.000,0.000
1,SRB.1.2_1,0.000000,224386.815369,2022,16626.023529,0.000,13.496
2,SRB.1.3_1,0.000000,115498.788230,2022,13804.294118,0.000,8.367
3,SRB.1.4_1,59207.660159,394397.557246,2022,28860.266667,2.052,13.666
4,SRB.2.1_1,0.000000,99159.145913,2022,6490.898039,0.000,15.277
...,...,...,...,...,...,...,...
156,SRB.25.6_1,0.000000,0.000000,2022,27803.866667,0.000,0.000
157,SRB.25.7_1,17371.204831,104737.223555,2022,17834.627451,0.974,5.873
158,SRB.25.8_1,32944.353096,178743.444409,2022,25052.764706,1.315,7.135
159,SRB.25.9_1,3094.980822,22431.926197,2022,24215.105882,0.128,0.926


In [11]:
flood.to_csv('D:\Work\WB\LDT\countries\SRB\datasets\SRB_flood_risk_2022.csv', index=False)

In [12]:
flood.describe()

,railway_length_flood_risk,road_length_flood_risk,year,population_sum,rail_length_flood_pcap,road_length_flood_pcap
count,161.000000,161.000000,161.0,161.000000,161.000000,161.000000
mean,1887.151055,54723.772493,2022.0,36592.626647,0.077857,2.065888
std,7030.069351,118466.351837,0.0,52658.708031,0.285249,5.121876
min,0.000000,0.000000,2022.0,937.603922,0.000000,0.000000
25%,0.000000,0.000000,2022.0,11576.000000,0.000000,0.000000
50%,0.000000,0.000000,2022.0,21012.349020,0.000000,0.000000
75%,0.000000,37662.880612,2022.0,39362.850980,0.000000,1.635000
max,59207.660159,678073.251347,2022.0,456825.266667,2.052000,40.251000
